In [40]:
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service


In [ ]:
def listToString(list1):
    str1 = " "
    return str1.join(list1)

chrome_driver_path = r"C:\Users\YSK\Documents\Drivers\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)
driver.maximize_window()

j= 2
house_list = []
while j<=18:
    driver.get(f"https://www.emlakjet.com/satilik-konut/kayseri-kocasinan/{j}/")
    i= 1
    j+= 1

    wait = WebDriverWait(driver, 30)
    while i<=29:
        if i in (3, 8, 9, 11, 14):
            i+=1
            continue
        details_dict = {}
        xpath = f'//*[@id="listing-search-wrapper"]/div[{str(i)}]'
        house_card = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        actions = ActionChains(driver)
        actions.move_to_element(house_card).click().perform()

        try:
            house_details = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_35T4WV')))
            actions = ActionChains(driver)
            actions.move_to_element(house_details[0]).click().perform()
        except:
            i+=1
            driver.execute_script("window.history.go(-1)")
            time.sleep(3)
            continue

        details = []
        for detail in house_details:
            details.append(detail.text)

        for item in details:
            if '\n' in item:
                key, value = item.split('\n', 1)
                details_dict[key] = value

        house_price = driver.find_element(By.CLASS_NAME, 'R-RKDB')
        details_dict['House Price'] = house_price.text

        house_location = driver.find_element(By.XPATH, '//div[@class="_3VQ1JB"]/p')
        details_dict['House Location'] = house_location.text

        house_list.append(details_dict)
        print(i,j)
        i = i+1
        driver.execute_script("window.history.go(-1)")
        time.sleep(3)

driver.quit()

In [47]:
df = pd.DataFrame(house_list)
df.to_csv('kocasinan_houses_data.csv')
print("CSV file created.")

CSV file created.
